In [ ]:
import psycopg2
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta

conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password)
conn.autocommit = True
%reload_ext sql_magic

In [ ]:
first_day_current_month =  datetime.today().replace(day=1)
last_day_last_month = first_day_current_month - timedelta(days=1)
first_day_last_month = last_day_last_month.replace(day=1).strftime("%Y-%m-%d")
last_day_last_month = last_day_last_month.strftime("%Y-%m-%d")
# parametry = f"data_wpr::date between '{first_day_last_month}' and '{last_day_last_month}'"

In [ ]:
%%read_sql df -c conn
select * from table_to_analyse;

In [ ]:
import nltk
from nltk import ngrams

raw_text = df.nazwaczynnosci.values

def create_ngram(n):
    print("Wyznaczanie gramów:", n)
    count_dictionary2 = {}
    for line in raw_text:
        token = nltk.word_tokenize(line)
        gram = list(ngrams(token, n)) 
        fdist = nltk.FreqDist(gram)
        for i,j in fdist.items():
            key = " ".join(i)
            if key in count_dictionary2.keys():
                count_dictionary2[key] += j #add how many times in this record 
            else:
                count_dictionary2[key] = j
    df_cnt = pd.DataFrame(count_dictionary2, index=[0])
    df_cnt = df_cnt.T
    df_cnt = df_cnt.reset_index()
    df_cnt.columns = ['Słowa', 'Cnt']
    df_cnt = df_cnt.sort_values(by='Cnt', ascending=False)
    df_cnt = df_cnt[df_cnt.Cnt>=100]
    return df_cnt

df_cnt4 = create_ngram(4)
df_cnt5 = create_ngram(5)
df_cnt6 = create_ngram(6)
    
with pd.ExcelWriter(f'n-gramy {first_day_last_month} - {last_day_last_month}.xlsx') as writer:  
    df_cnt4.to_excel(writer, sheet_name='4-gram', index=False)
    df_cnt5.to_excel(writer, sheet_name='5-gram', index=False)
    df_cnt6.to_excel(writer, sheet_name='6-gram', index=False)
